In [12]:
import pandas as pd
import json
import re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [13]:


data_name = "baseline_eval_only_random_sample"


data_name = "make_model_cictt"

#model_name = "baseline_lh_2"

#model_name = "baseline_lh_3"

model_name = "baseline_lh_0"
model_name = "baseline_lh_b"

model_name = "make_model_union"

run_name = data_name + "_model_" + model_name



# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))




In [14]:
import os
os.getcwd()

'/home/kbryan/ditto_aircraft_er/aircraft_er_notebooks'

In [15]:
df_all = pd.DataFrame(records)
print("number of rows in ground truth: ", len(df_all))
df_all.head()

number of rows in ground truth:  34679


,left,right,match,match_confidence
0,COL make VAL BOEING COL model VAL 727 COL cert...,COL make VAL GULFSTREAM COL model VAL GV COL s...,0,0.999995
1,COL make VAL DE HAVILLAND COL model VAL DH106 ...,COL make VAL KAMAN COL model VAL K225 COL seri...,0,0.999996
2,COL make VAL SIKORSKY COL model VAL S70 COL na...,COL make VAL CESSNA COL model VAL 180 COL seri...,0,0.999996
3,COL make VAL KLEMM COL model VAL L26 COL cert ...,COL make VAL BEECH COL model VAL 99 COL series...,0,0.999995
4,COL make VAL BALLOON WORKS COL model VAL FIREF...,COL make VAL BALLOON WORKS COL model VAL FIREF...,0,0.999991


In [16]:

gold_all = pd.read_csv("../data/ditto_aircraft/" + data_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])

gold_all_with_id = pd.read_csv("../data/ditto_aircraft/" + data_name + "/all_pairs_with_id.txt",  header=0)

print("number of rows in ground truth: ", len(gold_all))
print(gold_all.head())

print("number of rows in ground truth with id: ", len(gold_all_with_id))
print(gold_all_with_id.head())

number of rows in ground truth:  34679
                                                left  \
0  COL make VAL BOEING COL model VAL 727 COL cert...   
1  COL make VAL DE HAVILLAND COL model VAL DH106 ...   
2  COL make VAL SIKORSKY COL model VAL S70 COL na...   
3  COL make VAL KLEMM COL model VAL L26 COL cert ...   
4  COL make VAL BALLOON WORKS COL model VAL FIREF...   

                                               right  gold  
0  COL make VAL GULFSTREAM COL model VAL GV COL s...     0  
1  COL make VAL KAMAN COL model VAL K225 COL seri...     0  
2  COL make VAL CESSNA COL model VAL 180 COL seri...     0  
3  COL make VAL BEECH COL model VAL 99 COL series...     0  
4  COL make VAL BALLOON WORKS COL model VAL FIREF...     0  
number of rows in ground truth with id:  34679
                    left_id  right_id  \
0                BOEING 727      5620   
1        DE HAVILLAND DH106     21573   
2              SIKORSKY S70      3703   
3                 KLEMM L26      1322   
4  BAL

In [17]:
df_all["gold"] = gold_all_with_id["label"]
df_all["left_id"]= gold_all_with_id["left_id"]
df_all["right_id"]= gold_all_with_id["right_id"]


In [18]:
df_all.head()

,left,right,match,match_confidence,gold,left_id,right_id
0,COL make VAL BOEING COL model VAL 727 COL cert...,COL make VAL GULFSTREAM COL model VAL GV COL s...,0,0.999995,0,BOEING 727,5620
1,COL make VAL DE HAVILLAND COL model VAL DH106 ...,COL make VAL KAMAN COL model VAL K225 COL seri...,0,0.999996,0,DE HAVILLAND DH106,21573
2,COL make VAL SIKORSKY COL model VAL S70 COL na...,COL make VAL CESSNA COL model VAL 180 COL seri...,0,0.999996,0,SIKORSKY S70,3703
3,COL make VAL KLEMM COL model VAL L26 COL cert ...,COL make VAL BEECH COL model VAL 99 COL series...,0,0.999995,0,KLEMM L26,1322
4,COL make VAL BALLOON WORKS COL model VAL FIREF...,COL make VAL BALLOON WORKS COL model VAL FIREF...,0,0.999991,0,BALLOON WORKS FIREFLY 8B,1074


In [19]:
#All Data

y_true = df_all["gold"]
y_pred = df_all["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))



with open("../aircraft_er_predictions/" + run_name + "_metrics_all.txt", "w") as f:
    print("Accuracy:", accuracy_score(y_true, y_pred), file=f)
    print("\nClassification report:\n", file=f)
    print(classification_report(y_true, y_pred), file=f)
    print("\nConfusion matrix:\n", file=f)
    print(confusion_matrix(y_true, y_pred), file=f)

Accuracy: 0.99985582052539

Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     31227
           1       1.00      1.00      1.00      3452

    accuracy                           1.00     34679
   macro avg       1.00      1.00      1.00     34679
weighted avg       1.00      1.00      1.00     34679


Confusion matrix:
 [[31223     4]
 [    1  3451]]


In [8]:
errors = df_all[df_all["gold"] != df_all["match"]]
print(errors[["left","right","gold","match","match_confidence", "left_id", "right_id"]].head(20))

                                                  left  \
7           COL make VAL GULFSTREAM COL model VAL GVII   
23   COL make VAL BOMBARDIER COL model VAL CL600 2C...   
31   COL make VAL EMBRAER COL model VAL ERJ170 COL ...   
65   COL make VAL GULFSTREAM COL model VAL GVI COL ...   
83   COL make VAL SOCATA COL model VAL TBM700 COL n...   
91   COL make VAL BOMBARDIER COL model VAL CL600 2B...   
98   COL make VAL EMBRAER COL model VAL EMB550 COL ...   
102  COL make VAL CESSNA COL model VAL 206 COL name...   
187  COL make VAL EMBRAER COL model VAL EMB135 COL ...   
245         COL make VAL GULFSTREAM COL model VAL GVII   
258  COL make VAL CANADAIR COL model VAL CL600 2B16...   
319             COL make VAL SHORT COL model VAL SD330   
321  COL make VAL EMBRAER COL model VAL ERJ190 COL ...   
387         COL make VAL GULFSTREAM COL model VAL GVII   
403  COL make VAL BOMBARDIER COL model VAL BD100 CO...   
434  COL make VAL PIPER COL model VAL PA18S COL nam...   
473  COL make 

In [22]:
errors

,left,right,match,match_confidence,gold,left_id,right_id
340,COL make VAL BOEING COL model VAL 727 COL seri...,COL make VAL B COL model VAL 727 COL series VA...,1,0.274814,0,1816,B-727-290
343,COL make VAL BOEING COL model VAL 727 COL seri...,COL make VAL B COL model VAL 727 COL series VA...,1,0.594253,0,1816,B-727-295
3759,COL make VAL BOEING COL model VAL 747 COL seri...,COL make VAL B COL model VAL 747 COL series VA...,1,0.885383,0,2546,B-747-124
3760,COL make VAL BOEING COL model VAL 747 COL seri...,COL make VAL B COL model VAL 747 COL series VA...,1,0.872041,0,2546,B-747-127
3761,COL make VAL BOEING COL model VAL 747 COL seri...,COL make VAL B COL model VAL 747 COL series VA...,1,0.925247,0,2546,B-747-128
...,...,...,...,...,...,...,...
30806,COL make VAL CESSNA COL model VAL F172 COL ser...,COL make VAL REIMS COL model VAL F172 COL seri...,1,0.999679,0,3894,REIMS-F172-H
30807,COL make VAL CESSNA COL model VAL F172 COL ser...,COL make VAL REIMS COL model VAL F172 COL seri...,1,0.999313,0,3894,REIMS-F172-K
30808,COL make VAL CESSNA COL model VAL F172 COL ser...,COL make VAL REIMS COL model VAL F172 COL seri...,1,0.999637,0,3894,REIMS-F172-M
30809,COL make VAL CESSNA COL model VAL F172 COL ser...,COL make VAL REIMS COL model VAL F172 COL seri...,1,0.999683,0,3894,REIMS-F172-N


In [9]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [10]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "left_id": row["left_id"],
        "right_id": row["right_id"],
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "predicted": row["match"],
        "confidence": row["match_confidence"],
        "gold": row["gold"]
    })

aligned = pd.DataFrame(parsed)


In [11]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)

In [26]:
tmp = gold_all_with_id[gold_all_with_id["left_id"]==3571]

In [27]:
tmp["left"][100]

KeyError: 100